In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
drive.mount('/content/drive')


hot_100 = pd.read_csv("/content/drive/MyDrive/AnaDon/Hot_100.csv", sep = ',')
wasabi_albums = pd.read_csv("/content/drive/MyDrive/AnaDon/wasabi_albums.csv", sep = ',')
wasabi_artists = pd.read_csv("/content/drive/MyDrive/AnaDon/wasabi_artists.csv", sep = ',')
wasabi_songs = pd.read_csv("/content/drive/MyDrive/AnaDon/wasabi_songs.csv", sep = '\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-2039371051.py:7: DtypeWarning: Columns (5,27) have mixed types. Specify dtype option on import or set low_memory=False.
  wasabi_albums = pd.read_csv("/content/drive/MyDrive/AnaDon/wasabi_albums.csv", sep = ',')
/tmp/ipython-input-2039371051.py:8: DtypeWarning: Columns (3,41) have mixed types. Specify dtype option on import or set low_memory=False.
  wasabi_artists = pd.read_csv("/content/drive/MyDrive/AnaDon/wasabi_artists.csv", sep = ',')
/tmp/ipython-input-2039371051.py:9: DtypeWarning: Columns (15,17,20,41,42,43,44,50,67,74) have mixed types. Specify dtype option on import or set low_memory=False.
  wasabi_songs = pd.read_csv("/content/drive/MyDrive/AnaDon/wasabi_songs.csv", sep = '\t')


In [17]:
hot_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335195 entries, 0 to 335194
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   chart_position     335195 non-null  int64  
 1   chart_date         335195 non-null  object 
 2   song               335195 non-null  object 
 3   performer          335195 non-null  object 
 4   song_id            335195 non-null  object 
 5   instance           335195 non-null  float64
 6   time_on_chart      335195 non-null  int64  
 7   consecutive_weeks  302067 non-null  float64
 8   previous_week      302067 non-null  float64
 9   peak_position      335195 non-null  int64  
 10  worst_position     335195 non-null  int64  
 11  chart_debut        335195 non-null  object 
 12  chart_url          335195 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 33.2+ MB


In [ ]:
wasabi_albums.head()

In [ ]:
wasabi_artists.head()

In [ ]:
wasabi_songs.info()

In [ ]:
colonnes_a_garder = [
    # === IDENTIFIANTS DE BASE ===
    '_id',              # ID unique de la chanson (identifiant interne Wasabi)
    'title',            # Titre de la chanson (clé principale pour la fusion)
    'artist',           # Artiste (clé secondaire pour la fusion)
    'albumTitle',       # Nom de l’album d’origine

    # === CARACTÉRISTIQUES AUDIO (pour Q1 - Formule du hit) ===
    'arousal',          # Niveau d’énergie émotionnelle perçue
    'valence',          # Niveau de positivité émotionnelle
    'bpm',              # Tempo (battements par minute)
    'length',           # Durée du morceau (en secondes)

    # === MÉTADONNÉES STRUCTURELLES ===
    'position',         # Numéro du titre dans l’album (pour Q3)
    'genre',            # Genre musical principal
    'album_genre',      # Genre de l’album (utile si différent du morceau)

    # === INFORMATIONS TEMPORELLES (pour Q2 - Cycles d’évolution) ===
    'publicationDate',  # Date de publication initiale
    'releaseDate',      # Date de sortie (souvent plus fiable)

    # === AUTRES FEATURES PERTINENTES ===
    'language',         # Langue du morceau (anglais, espagnol, etc.)
    'isClassic',        # Indique si le morceau est considéré comme un “classique”
    'explicit_content_lyrics',  # Niveau d’explicit des paroles
    'recordLabel',      # Maison de disque (majors vs indépendants)
    'writer',           # Auteur / compositeur principal
    'producer',         # Producteur (utile pour repérer les styles à succès)
]


# Filtrer pour garder seulement les colonnes qui existent
colonnes_existantes = [col for col in colonnes_a_garder if col in wasabi_songs.columns]

print(f"Colonnes sélectionnées ({len(colonnes_existantes)}):")
for col in colonnes_existantes:
    print(f"{col}")

# Créer le dataframe nettoyé
wasabi_clean_song = wasabi_songs[colonnes_existantes].copy()

print(f"\nDataFrame nettoyé créé: {wasabi_clean_song.shape}")

print("\nValeurs manquantes:")
wasabi_clean_song.isnull().sum()

In [ ]:
# Préparer Wasabi
wasabi_clean_song['source'] = 'wasabi'
wasabi_clean_song['song_title'] = wasabi_clean_song['title']
wasabi_clean_song['artist_name'] = wasabi_clean_song['artist']

# Préparer Hot 100
hot_100_prepared = hot_100.copy()
hot_100_prepared['source'] = 'hot_100'
hot_100_prepared['song_title'] = hot_100_prepared['song']
hot_100_prepared['artist_name'] = hot_100_prepared['performer']

# Créer une clé de jointure commune
wasabi_clean_song['jointure_key'] = (
    wasabi_clean_song['song_title'].str.lower().str.strip() + "|" +
    wasabi_clean_song['artist_name'].str.lower().str.strip()
)

hot_100_prepared['jointure_key'] = (
    hot_100_prepared['song_title'].str.lower().str.strip() + "|" +
    hot_100_prepared['artist_name'].str.lower().str.strip()
)

In [ ]:
# Trouver les chansons Hot 100 qui ne sont PAS dans Wasabi
hot_100_uniques = hot_100_prepared[~hot_100_prepared['jointure_key'].isin(wasabi_clean['jointure_key'])]

print(f"Statistiques de fusion :")
print(f"Chansons Wasabi : {len(wasabi_clean)}")
print(f"Chansons Hot 100 totales : {len(hot_100_prepared)}")
print(f"Chansons Hot 100 UNIQUES (pas dans Wasabi) : {len(hot_100_uniques)}")
print(f"Doublons (déjà dans Wasabi) : {len(hot_100_prepared) - len(hot_100_uniques)}")

In [ ]:
pip install unidecode

In [ ]:
import unidecode

def clean_text(x):
    if pd.isna(x):
        return ""
    x = str(x).lower().strip()
    x = unidecode.unidecode(x)
    x = x.replace("&", "and").replace("feat.", "").replace("featuring", "")
    x = x.replace("'", "").replace("-", " ").replace("(", "").replace(")", "")
    return " ".join(x.split())

# Utiliser les NOUVELLES colonnes que tu as créées
if "title_clean" not in wasabi_clean_song.columns:
    wasabi_clean_song["title_clean"] = wasabi_clean_song["song_title"].apply(clean_text)  # ← Changé "title" → "song_title"
    wasabi_clean_song["artist_clean"] = wasabi_clean_song["artist_name"].apply(clean_text)  # ← Changé "artist" → "artist_name"

if "title_clean" not in hot_100_prepared.columns:
    hot_100_prepared["title_clean"] = hot_100_prepared["song_title"].apply(clean_text)  # ← Changé "title" → "song_title"
    hot_100_prepared["artist_clean"] = hot_100_prepared["artist_name"].apply(clean_text)  # ← Changé "artist" → "artist_name"

In [ ]:
merged = pd.merge(
    hot_100_prepared,
    wasabi_clean_song,
    on="jointure_key",
    how="inner",
    suffixes=("_hot100", "_wasabi")
)

print(f"Fusion effectuée : {len(merged)} lignes communes trouvées")

In [ ]:
wasabi_clean_song.head(2)

In [ ]:
hot_100.isna().sum()

In [ ]:
wasabi_albums.isna().sum()

In [ ]:
wasabi_artists.isna().sum()

In [ ]:
wasabi_songs.isna().sum()